In [1]:
import asyncio
import time
from datetime import datetime
import pandas as pd

In [ ]:
from client_utils import ClientSetup

totp = "360359"
setup = ClientSetup()
client = setup.login(totp)
instrument_tokens = [{"instrument_token": "Nifty 50", "exchange_segment": "nse_cm"},]
print("Nifty Current Price: ",client.quotes(instrument_tokens = instrument_tokens, quote_type = "all")[0]['ltp'])

In [3]:

csv_data = client.scrip_master(exchange_segment="nse_fo")
df = pd.read_csv(csv_data)
df = df[df["pSymbolName"].str.match(r"(?i)^NIFTY$", na=False)]
df.to_csv("nifty-options.csv", index=False)

In [10]:
def get_nifty_option_symbol(pScripRefKey):
    return df[df["pScripRefKey"] == pScripRefKey]["pTrdSymbol"].values[0]

def nifty_symbol(date, strike=None, typ="CE"):
    d = datetime.strptime(date, "%d-%m-%Y") if isinstance(date, str) else date
    strike_part = "" if typ.upper() == "FUT" else f"{float(strike):.2f}"
    symbol_string =  f"NIFTY{d.strftime('%d%b%y').upper()}{strike_part}{typ.upper()}"
    return get_nifty_option_symbol(symbol_string)


In [70]:
def cancel_order(order_id, amo="YES"):
    status = client.cancel_order(order_id=order_id, amo=amo)
    print("Order Cancel Status: ", status)
    return status

In [61]:
def extract_order_status(order_id):
    """
    Returns normalized status: COMPLETE / OPEN / CANCELLED / REJECTED
    """
    order_history_response = client.order_history(order_id)
    order = order_history_response["data"]["data"][0]
    raw_status = order.get("ordSt", "").lower()

    if any(x in raw_status for x in ["complete", "traded"]):
        return "COMPLETE"

    if any(x in raw_status for x in ["cancel", "expired"]):
        return "CANCELLED"

    if "reject" in raw_status:
        return "REJECTED"

    return "OPEN"



In [67]:
lot_size = 75

def place_order(trade_option,quantity, price, transaction_type="B", amo="YES"):
    #Safe checks for price
    if transaction_type=="B" and price == None:
        price = 1
    if transaction_type=="S" and price == None:
        price = 1000
    
    order = client.place_order(
        exchange_segment="nse_fo",
        product="NRML",
        price=str(price),                    # Limit price (must be valid)
        order_type="L",               # AMO must be LIMIT
        quantity=str(lot_size*quantity),         # Lot size for NIFTY options is 75
        trading_symbol=trade_option,
        transaction_type=transaction_type,
        amo=amo,                    # 🔑 IMPORTANT: Enable AMO
        disclosed_quantity="0",
        trigger_price="0",
        validity="DAY"
    )
    return order['nOrdNo']

In [63]:
def wait_for_execution(order_id, timeout):
    start = time.time()
    while time.time() - start < timeout:
        status = extract_order_status(order_id)  # <-- your function
        if status == "COMPLETE":
            return True
        elif status in ("CANCELLED", "REJECTED"):
            return False
        time.sleep(1)  # poll every 1 second
    return False

In [ ]:
# trade_option = nifty_symbol("30-12-2025", 26000, "CE")
trade_option = nifty_symbol("30-12-2025", 26000, "CE")
buy_price = 1
buy_time = 10

sell_price = 2
sell_time = 15
# Buy Order
buy_order_id = place_order(trade_option,quantity=1, price=buy_price, transaction_type="B")

if not wait_for_execution(buy_order_id, buy_time):
    cancel_order(order_id=buy_order_id)
    print("❌ Buy not executed in time")

else:
    print("✅ Buy executed")
    # Sell Order
    sell_order_id = place_order(trade_option,quantity=1, price=sell_price, transaction_type="S")
    if wait_for_execution(sell_order_id, sell_time):
        print("✅ Sell executed")
    else:
        client.cancel_order(order_id=sell_order_id)
        print("❌ Sell not executed in time")
        print("Advise manual sell at market price")

Order Cancel Status:  {'result': '251225000003323', 'stat': 'Ok', 'stCode': 200}
❌ Buy not executed in time


## Buying Strategy

In [ ]:
def buy_first_then_sell(trade_option, buy_price, buy_time, sell_price, sell_time):
    buy_order_id = place_order(trade_option,quantity=1, price=buy_price, transaction_type="B")

    if not wait_for_execution(buy_order_id, buy_time):
        cancel_order(order_id=buy_order_id)
        print("❌ Buy not executed in time")
        return False

    print("✅ Buy executed")
    # Sell Order
    sell_order_id = place_order(trade_option,quantity=1, price=sell_price, transaction_type="S")
    if wait_for_execution(sell_order_id, sell_time):
        print("✅ Sell executed")
        return True
    cancel_order(order_id=sell_order_id)
    print("❌ Sell not executed in time")
    print("Advise manual sell at market price")
    return False

In [ ]:
# trade_option = nifty_symbol("30-12-2025", 26000, "CE")
trade_option = nifty_symbol("30-12-2025", 26000, "CE")
buy_price = 1
buy_time = 10
sell_price = 2
sell_time = 15
result = buy_first_then_sell(trade_option, buy_price, buy_time, sell_price, sell_time)
print(result)

## Selling Strategy

In [72]:
def sell_first_then_buy(trade_option, sell_price, sell_time, buy_price, buy_time):
    # Sell Order first
    sell_order_id = place_order(trade_option, quantity=1, price=sell_price, transaction_type="S")

    if not wait_for_execution(sell_order_id, sell_time):
        cancel_order(order_id=sell_order_id)
        print("❌ Sell not executed in time")
        return False

    print("✅ Sell executed")

    # Buy Order
    buy_order_id = place_order(
        trade_option,
        quantity=1,
        price=buy_price,
        transaction_type="B"
    )

    if wait_for_execution(buy_order_id, buy_time):
        print("✅ Buy executed")
        return True

    cancel_order(order_id=buy_order_id)
    print("❌ Buy not executed in time")
    print("Advise manual buy at market price")
    return False


In [73]:
# trade_option = nifty_symbol("30-12-2025", 26000, "CE")
trade_option = nifty_symbol("30-12-2025", 26000, "CE")
buy_price = 1
buy_time = 10
sell_price = 2
sell_time = 15
result = sell_first_then_buy(trade_option, sell_price, sell_time, buy_price, buy_time)
print(result)

Order Cancel Status:  {'result': '251225000003886', 'stat': 'Ok', 'stCode': 200}
❌ Sell not executed in time
False


In [25]:
order_id = "251225000002911"  # Replace with your actual order ID
status = extract_order_status(order_id)
print(status)

OPEN
